In [1]:
from dataclasses import dataclass
import numpy as np
from deck import full_euchre_deck
from branch_calc import (
    filter_branch_by_hand,
    setdiff2d_idx,
    suit_id,
    find_winner,
    calc_all_possible_hands,
)
from dealer import Dealer

In [2]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

In [3]:
@dataclass
class PlayRound:
    hands: list
    winner: int

    def play_round(self, bit: int):
        all_possible_tricks = calc_all_possible_hands(hands=self.hands)
        len(all_possible_tricks)
        contains_target = np.any(
            np.all(all_possible_tricks == self.hands[self.winner][bit], axis=-1), axis=1
        )
        stump = all_possible_tricks[contains_target]
        stump = filter_branch_by_hand(stump, self.hands[(self.winner+1)%4], (self.winner+1)%4, self.hands[self.winner][bit])
        stump = filter_branch_by_hand(stump, self.hands[(self.winner+2)%4], (self.winner+2)%4, self.hands[self.winner][bit])
        stump = filter_branch_by_hand(stump, self.hands[(self.winner+3)%4], (self.winner+3)%4, self.hands[self.winner][bit])
        self.round_winner = find_winner(
            lead=stump[bit][self.winner], trick=stump[bit]
        )  # will need to fix this in order to iterate through all hands
        self.round_winning_team = (
            find_winner(lead=stump[bit][self.winner], trick=stump[bit]) % 2
        )  # will need to fix this in order to iterate through all hands
        self.next_round_hands = [
            setdiff2d_idx(self.hands[0], stump[bit]),
            setdiff2d_idx(self.hands[1], stump[bit]),
            setdiff2d_idx(self.hands[2], stump[bit]),
            setdiff2d_idx(self.hands[3], stump[bit]),
        ]

        return (
            stump[bit],
            self.round_winner,
            self.round_winning_team,
            np.array(self.next_round_hands),
        )


In [4]:
round1 = PlayRound(hands=hands5, winner=0)
round1.play_round(bit=0)
round2 = PlayRound(hands=round1.next_round_hands, winner=round1.round_winner)
round2.play_round(bit=0)
round3 = PlayRound(hands=round2.next_round_hands, winner=round2.round_winner)
round3.play_round(bit=0)
round4 = PlayRound(hands=round3.next_round_hands, winner=round3.round_winner)
round4.play_round(bit=0)
round5 = PlayRound(hands=round4.next_round_hands, winner=round4.round_winner)
round5.play_round(bit=0)



(array([[ -9,   0],
        [  0, -11],
        [-14,   0],
        [ 12,   0]]),
 2,
 0,
 array([], shape=(4, 0, 2), dtype=int32))